In this notebook we will generate and save the data frames that we will use in visualization, using the clean ones done in the previous steps

In [27]:
import pandas as pd
sys.path.append('../')
import src.limpieza as lmp
from geopy.geocoders import Nominatim
import re
import geopandas as gpd
from keplergl import KeplerGl

In [28]:
df_civitatis_clean = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_clean.pkl')
df_turistas_admon = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\Barcelona_cleaned_tourist.pkl')


In [2]:
df_civitatis_location = pd.read_csv(r"C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_location.csv", sep=',', encoding='latin-1')

# Civitatis

Total number of tourist grouped by month

In [ ]:
lmp.total_civitatis_tourists_month(df_civitatis_clean)

In [15]:
df_civitatis_clean.shape

(16231, 5)

In [3]:
df_civitatis_clean.head(5)

,fecha,nombre,procedencia,opinion,viajo_con
0,2022-03-11,Ana,"Madrid, España","Victor es encantador, te explica todo al detal...",Viajó con hijos mayores
1,2022-03-11,Montserrat,España,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo
2,2022-03-11,Ainhoa,España,,Viajó en pareja
3,2022-03-11,Patricia Suárez Carrillo,"Madrid, España",A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja
4,2022-03-11,Anónimo,,,


Longitude and latitude of the place of origin of tourists

In [4]:
def extraer_coordenadas(x):
    try:
        geolocator = Nominatim(user_agent = "pepe")
        location = geolocator.geocode(x)

        return location.latitude, location.longitude
    except:
        return 'no data'

In [5]:
df_coordinates = pd.DataFrame(df_civitatis_clean['procedencia'].unique())

In [ ]:
df_coordinates["georeferenced_column"] = df_coordinates[0].apply(extraer_coordenadas)

In [10]:
df_coordinates.to_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\lat_lon.pkl')


In [11]:
df_coordinates = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\lat_lon.pkl')

In [12]:
df_coordinates.head(5)

,0,georeferenced_column,latitude,longitude
0,"Madrid, España","(40.4167047, -3.7035825)",40.416705,-3.703582
1,España,"(39.3260685, -4.8379791)",39.326068,-4.837979
2,,no data,NaN,NaN
3,"Alicante, España","(38.3436365, -0.4881708)",38.343637,-0.488171
4,"Quito, Ecuador","(-0.2201641, -78.5123274)",-0.220164,-78.512327


In [8]:
for i in range(len(df_coordinates)):
    df_coordinates['latitude'].iloc[i] = df_coordinates['georeferenced_column'].iloc[i][0]
    df_coordinates['longitude'].iloc[i] = df_coordinates['georeferenced_column'].iloc[i][1]
df_coordinates['latitude'] = pd.to_numeric(df_coordinates['latitude'],errors='coerce')
df_coordinates['longitude'] = pd.to_numeric(df_coordinates['longitude'],errors='coerce')


C:\Users\mituc\AppData\Local\Temp/ipykernel_5692/2326016387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coordinates['latitude'].iloc[i] = df_coordinates['georeferenced_column'].iloc[i][0]
C:\Users\mituc\AppData\Local\Temp/ipykernel_5692/2326016387.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coordinates['longitude'].iloc[i] = df_coordinates['georeferenced_column'].iloc[i][1]


In [17]:
df_civitatis_clean.head(5)

,fecha,nombre,procedencia,opinion,viajo_con
0,2022-03-11,Ana,"Madrid, España","Victor es encantador, te explica todo al detal...",Viajó con hijos mayores
1,2022-03-11,Montserrat,España,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo
2,2022-03-11,Ainhoa,España,,Viajó en pareja
3,2022-03-11,Patricia Suárez Carrillo,"Madrid, España",A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja
4,2022-03-11,Anónimo,,,


In [13]:
df_civitatis_clean.set_index(['procedencia'], inplace=True)
df_coordinates.set_index([0], inplace=True)

In [14]:
df_civitatis_location = df_civitatis_clean.join(df_coordinates, on='procedencia')

In [21]:
geolocator = Nominatim(user_agent="miguel")
location = geolocator.geocode("Barcelona, España")
df_civitatis_location['bar_lat'] = location.latitude
df_civitatis_location['bar_lon'] = location.longitude

In [16]:
df_civitatis_location['fecha'] = df_civitatis_location['fecha'].astype(str) + ' 00:00'

In [17]:
df_civitatis_location['fecha'] = pd.to_datetime(df_civitatis_location['fecha'], format='%Y-%m-%d %H:%M') 

In [18]:
df_civitatis_location['fecha']=df_civitatis_location['fecha'].dt.strftime('%Y-%m-%d %H:%M')

In [267]:
df_civitatis_location.dtypes

procedencia              object
fecha                    object
nombre                   object
opinion                  object
viajo_con                object
georeferenced_column     object
latitude                float64
longitude               float64
bar_lat                 float64
bar_lon                 float64
dtype: object

In [23]:
df_civitatis_location.to_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_location.csv')

In [24]:
df_civitatis_location = pd.read_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_location.csv')

In [268]:
map_1 = KeplerGl(height = 600, weight = 500, data = {"viajes": df_civitatis_location})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [273]:
map_1.save_to_html(file_name='civitatis.html')

Map saved to civitatis.html!


In [269]:
map_1

KeplerGl(data={'viajes':                     procedencia             fecha  \
0               Madrid, EspaÃ±a …

In [26]:
df_civitatis_location.head(5)

,procedencia,fecha,nombre,opinion,viajo_con,georeferenced_column,latitude,longitude,bar_lat,bar_lon
0,"Madrid, España",2022-03-11 00:00,Ana,"Victor es encantador, te explica todo al detal...",Viajó con hijos mayores,"(40.4167047, -3.7035825)",40.416705,-3.703582,41.382894,2.177432
1,España,2022-03-11 00:00,Montserrat,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo,"(39.3260685, -4.8379791)",39.326068,-4.837979,41.382894,2.177432
2,España,2022-03-11 00:00,Ainhoa,NaN,Viajó en pareja,"(39.3260685, -4.8379791)",39.326068,-4.837979,41.382894,2.177432
3,"Madrid, España",2022-03-11 00:00,Patricia Suárez Carrillo,A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja,"(40.4167047, -3.7035825)",40.416705,-3.703582,41.382894,2.177432
4,NaN,2022-03-11 00:00,Anónimo,NaN,NaN,no data,NaN,NaN,41.382894,2.177432


In [4]:
df_civitatis_location.drop(['Unnamed: 0'], axis=1, inplace=True)

# Spanish goverment

As civitatis started its activity for the tour that we are studying in June 2017, we'll select that period from the data frame we had generated using the data given by the goverment.


In [30]:
lmp.admon_cividates(df_turistas_admon)

In [31]:
df_turistas_admon_17 = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\admon_cividates.pkl')


In [32]:
df_turistas_admon_17.tail(5)

,periodo,total_viajeros
5,2021-09,411625
4,2021-10,494753
3,2021-11,435425
2,2021-12,394432
1,2022-01,301474
